## **Power BI - A Ponta do Pipeline**

### Parte 1: Preparação dos Dados

Vamos identificar os itens que estão abaixo do Q1 (Quartil 1) e acima do Q3 (Quartil 3), criar uma etiqueta (rótulo/'label') para eles e exportar.

Focaremos, devido ao contexto dos dados de aulas anteriores, em: "Municípios com repasse médio Muito Alto ou Muito Baixo do benefício social".

#### Código de Preparação e Exportação

Executamos este código neste notebook para gerar o arquivo que o Power BI vai ler. Vamos usar o DataFrame agregado por município que criamos na etapa anterior:

In [1]:
import polars as pl

In [8]:
inicial = (
    pl.scan_parquet("../Projeto/orders.parquet")
    .with_columns(
        pl.col("OrderDate")
        .str.strptime(pl.Date, "%Y-%m-%d", strict=False)
        .dt.to_string("%Y-%m")  # Formata como string YYYY-MM
        .alias("DATA_REF")
    )
)

resultado = inicial.collect()
display(resultado)

OrderID,OrderDate,CustomerID,ProductID,SellerID,Quantity,Discount,Tax,ShippingCost,TotalAmount,PaymentMethod,OrderStatus,DATA_REF
str,str,str,str,str,i64,f64,f64,f64,f64,str,str,str
"""ORD0000001""","""2023-01-31""","""CUST001504""","""P00014""","""SELL01967""",3,0.0,0.0,0.09,319.86,"""Debit Card""","""Delivered""","""2023-01"""
"""ORD0000002""","""2023-12-30""","""CUST000178""","""P00040""","""SELL01298""",1,0.05,19.1,1.74,259.64,"""Amazon Pay""","""Delivered""","""2023-12"""
"""ORD0000003""","""2022-05-10""","""CUST047516""","""P00044""","""SELL00908""",3,0.1,7.57,5.91,108.06,"""Debit Card""","""Delivered""","""2022-05"""
"""ORD0000004""","""2023-07-18""","""CUST030059""","""P00041""","""SELL01164""",5,0.15,11.42,5.53,159.66,"""Cash on Delivery""","""Delivered""","""2023-07"""
"""ORD0000005""","""2023-02-04""","""CUST048677""","""P00029""","""SELL01411""",2,0.25,38.67,9.23,821.36,"""Credit Card""","""Cancelled""","""2023-02"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""ORD0000685""","""2022-10-17""","""CUST030151""","""P00009""","""SELL01296""",2,0.0,24.58,10.82,342.64,"""Credit Card""","""Shipped""","""2022-10"""
"""ORD0000686""","""2022-01-30""","""CUST029666""","""P00028""","""SELL00476""",1,0.05,42.91,0.25,281.53,"""Cash on Delivery""","""Delivered""","""2022-01"""
"""ORD0000687""","""2021-11-06""","""CUST041687""","""P00017""","""SELL00214""",1,0.3,6.62,14.37,153.42,"""Credit Card""","""Delivered""","""2021-11"""


In [12]:
# 1. Calcular os limites (Q1 e Q3) baseados no Valor Médio por Município
# (Assumindo que 'df_Vendas' já existe da etapa anterior. Se não, recrie-o agrupando o LazyFrame)
df_Vendas = inicial.group_by("DATA_REF").agg(pl.col("TotalAmount").mean().alias("Valor_Medio_Pago"))

# Precisamos coletar para calcular os quartis exatos da distribuição das cidades
df_agregado = df_Vendas.collect()

q1 = df_agregado.select(pl.col("Valor_Medio_Pago").quantile(0.25)).item()
q3 = df_agregado.select(pl.col("Valor_Medio_Pago").quantile(0.75)).item()

print(f"Meses com vendas abaixo de R$ {q1:.2f} são 'Menor faturamento'")
print(f"Meses com vendas acima de R$ {q3:.2f} são 'Maior Faturamento'")

# 2. Criar a coluna "Etiqueta" (Classificação) e Filtrar apenas os extremos
# Conforme o roteiro: "Concatenamos os resultados adicionando uma coluna nova 'etiqueta' para indicar se o município é 'Alto Repasse' ou 'Baixo Repasse'."
df_export = df_agregado.with_columns(
    pl.when(pl.col("Valor_Medio_Pago") > q3).then(pl.lit("Maior Faturamento"))
    .when(pl.col("Valor_Medio_Pago") < q1).then(pl.lit("Menor faturamento"))
    .otherwise(pl.lit("Médio")) # Apenas para preencher, mas vamos filtrar
    .alias("Etiqueta_Classificacao")
).filter(
    (pl.col("Etiqueta_Classificacao") == "Maior Faturamento") | 
    (pl.col("Etiqueta_Classificacao") == "Menor faturamento")
)

# 3. Exportar para CSV (Simulando a entrega para o time de Dataviz)
df_export.write_csv("VendasporMes.csv")

print("Arquivo 'VendasporMes.csv' gerado com sucesso!")
print(df_export.head())

Meses com vendas abaixo de R$ 790.12 são 'Menor faturamento'
Meses com vendas acima de R$ 1065.83 são 'Maior Faturamento'
Arquivo 'VendasporMes.csv' gerado com sucesso!
shape: (5, 3)
┌──────────┬──────────────────┬────────────────────────┐
│ DATA_REF ┆ Valor_Medio_Pago ┆ Etiqueta_Classificacao │
│ ---      ┆ ---              ┆ ---                    │
│ str      ┆ f64              ┆ str                    │
╞══════════╪══════════════════╪════════════════════════╡
│ 2023-04  ┆ 638.986364       ┆ Menor faturamento      │
│ 2024-03  ┆ 1087.125455      ┆ Maior Faturamento      │
│ 2024-09  ┆ 772.376154       ┆ Menor faturamento      │
│ 2023-07  ┆ 782.505          ┆ Menor faturamento      │
│ 2023-11  ┆ 1084.979167      ┆ Maior Faturamento      │
└──────────┴──────────────────┴────────────────────────┘


### Parte 2: Introdução ao Power BI

Agora sim! Saímos do Python e abrimos o Power BI Desktop. O que você precisa saber:

- **O Pipeline de Dados**: Você atuou como Engenheiro/Cientista limpando o dado "sujo" (Parquet/CSV bruto) no Python. O Power BI é a última milha dessa maratona, onde o gestor tomará decisões no que for demonstrado.

- **Power BI não é ETL!**: Embora essa ferramenta tenha o Power Query, não devemos usá-lo para processar Terabytes de dados. O processamento pesado fica no Python com a assistência de bibliotecas dedicadas (no caso, Polars). O Power BI deve receber o dado já "mastigado" (como fizemos no CSV acima).


- **DataViz**: O foco aqui é contar uma história (Storytelling) com os dados.

### Parte 3: Conectando e Modelando

Após abrir o Power BI Desktop, vamos seguir pelos passos abaixo:

1. Clique em Obter Dados > Texto/CSV;

2. Selecione o arquivo bolsa_familia_extremos_municipios.csv que acabamos de gerar (no seu caso, para o projeto, o arquivo CSV específico da sua fonte de dados);

3. Clique em Transformar Dados para abrir o Power Query;

4. **Verificação**: Confira se os tipos de dados estão corretos (Valor como decimal, Município como texto etc). Como já limpamos previamente no Polars, deve estar tudo certo! Clique em "Fechar e Aplicar";

5. **Modelagem de Dados (Star Schema)**: Como temos apenas uma tabela de fatos (neste caso de exemplo, os pagamentos), vamos criar uma tabela auxiliar para simular um cenário real de modelagem. Vá na aba Modelagem > Nova Tabela;

6. Vamos criar uma tabela de Regiões usando DAX simples para relacionar com a coluna UF dos nossos dados:

In [64]:
'''
Tabela_Regioes = DATATABLE(
    "UF", STRING, "Regiao", STRING,
    {
        -- Norte
        {"AC", "Norte"},
        {"AP", "Norte"},
        {"AM", "Norte"},
        {"PA", "Norte"},
        {"RO", "Norte"},
        {"RR", "Norte"},
        {"TO", "Norte"},

        -- Nordeste
        {"AL", "Nordeste"},
        {"BA", "Nordeste"},
        {"CE", "Nordeste"},
        {"MA", "Nordeste"},
        {"PB", "Nordeste"},
        {"PE", "Nordeste"},
        {"PI", "Nordeste"},
        {"RN", "Nordeste"},
        {"SE", "Nordeste"},

        -- Centro-Oeste
        {"DF", "Centro-Oeste"},
        {"GO", "Centro-Oeste"},
        {"MT", "Centro-Oeste"},
        {"MS", "Centro-Oeste"},

        -- Sudeste
        {"ES", "Sudeste"},
        {"MG", "Sudeste"},
        {"RJ", "Sudeste"},
        {"SP", "Sudeste"},

        -- Sul
        {"PR", "Sul"},
        {"RS", "Sul"},
        {"SC", "Sul"}
    }
)
'''

'\nTabela_Regioes = DATATABLE(\n    "UF", STRING, "Regiao", STRING,\n    {\n        -- Norte\n        {"AC", "Norte"},\n        {"AP", "Norte"},\n        {"AM", "Norte"},\n        {"PA", "Norte"},\n        {"RO", "Norte"},\n        {"RR", "Norte"},\n        {"TO", "Norte"},\n\n        -- Nordeste\n        {"AL", "Nordeste"},\n        {"BA", "Nordeste"},\n        {"CE", "Nordeste"},\n        {"MA", "Nordeste"},\n        {"PB", "Nordeste"},\n        {"PE", "Nordeste"},\n        {"PI", "Nordeste"},\n        {"RN", "Nordeste"},\n        {"SE", "Nordeste"},\n\n        -- Centro-Oeste\n        {"DF", "Centro-Oeste"},\n        {"GO", "Centro-Oeste"},\n        {"MT", "Centro-Oeste"},\n        {"MS", "Centro-Oeste"},\n\n        -- Sudeste\n        {"ES", "Sudeste"},\n        {"MG", "Sudeste"},\n        {"RJ", "Sudeste"},\n        {"SP", "Sudeste"},\n\n        -- Sul\n        {"PR", "Sul"},\n        {"RS", "Sul"},\n        {"SC", "Sul"}\n    }\n)\n'

7. Vá para a Exibição de Modelo (ícone de diagrama à esquerda) e arraste o campo UF da tabela bolsa_familia para o campo UF da Tabela_Regioes.

8. Finalizado! Neste nosso exemplo foi criado um relacionamento Um-para-Muitos (1-N).

### Parte 4: DAX

Para entendermos a linguagem de análise DAX, praticaremos CALCULATE e SUM. Vamos criar medidas para analisar nossos dados de "Extremos".

1. Indo na aba Relatório, clique com o botão direito na sua tabela de dados e selecione *Nova Medida*;

2. Vamos calcular a média simples dos dados que importamos:

In [65]:
'''
Media_Valor_Extremos = AVERAGE('bolsa_familia_extremos_municipios'[Valor_Medio_Pago])
'''

"\nMedia_Valor_Extremos = AVERAGE('bolsa_familia_extremos_municipios'[Valor_Medio_Pago])\n"

3. Depois, criamos outro medida para calcular a média apenas das cidades classificadas como "Alto Repasse", ignorando os filtros de tela:

In [66]:
'''
Media_Alto_Repasse = 
CALCULATE(
    [Media_Valor_Extremos],
    'bolsa_familia_extremos_municipios'[Etiqueta_Classificacao] = "Alto Repasse"
)
'''

'\nMedia_Alto_Repasse = \nCALCULATE(\n    [Media_Valor_Extremos],\n    \'bolsa_familia_extremos_municipios\'[Etiqueta_Classificacao] = "Alto Repasse"\n)\n'

4. Mais uma medida - Total de Cidades Analisadas:

In [67]:
'''
Total_Cidades = COUNTROWS('bolsa_familia_extremos_municipios')
'''

"\nTotal_Cidades = COUNTROWS('bolsa_familia_extremos_municipios')\n"

### Parte 5: Visualização e Análise

Finalmente podemos responder algumas perguntas pelo Dataviz:


1. **Matriz ou Tabela**:

- Coloque NOME_MUNICIPIO nas linhas.

- Coloque Valor_Medio_Pago nos Valores e Etiqueta_Classificacao nas colunas.

- *Objetivo*: Ver lista das cidades e se elas são "Alto" ou "Baixo".


2. **Segmentação de Dados (Filtro)**:

- Use a coluna Regiao da nossa tabela auxiliar (Tabela_Regioes).

- Use também a coluna Etiqueta_Classificacao como filtro.

- *Teste*: Clique em "Sudeste" e veja apenas as cidades extremas do Sudeste.


3. **Cartões (KPIs)**:

- Arraste as medidas Media_Alto_Repasse e Total_Cidades para cartões separados.

- Selecione no filtro a Etiqueta "Alto Repasse".

- Observe quais regiões possuem mais cidades com tickets médios altos do Bolsa Família.

- Existe algum padrão geográfico? (Ex: Cidades do Norte/Nordeste aparecem mais/menos no topo ou cidades do Sul/Sudeste?).